## Задача 3
### Часть 1

Нужно написать скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков.

Входная точка по [ссылке](http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227). 
Затем нужно перейти на сайты региональных избирательных комиссий. 

Результаты нужно сохранить в cvs-файл, sqlite базе данных или parquet-файле. 

В итоге должна получиться таблица с полями (headers): 
- название региона: regions_names
- название ТИК (территориальная избирательная комиссия): TIKs_names
- номер УИК (участковая избирательная комиссия): UIK_names
- 20 стандартных полей из итогового протокола: UIK_statistics 

In [95]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd 
import csv

In [2]:
start_link = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'

In [3]:
start_req = requests.get(start_link)

In [4]:
start_soup = BeautifulSoup(start_req.text, 'lxml')

### Названия полей в таблице

In [5]:
headers = ['Регион', 'ТИК', 'УИК']

In [6]:
def get_standard_fields_titles(soup):
    standard_fields = []
    standard_fields_soup = soup.findAll('table')[1].findAll('table')[6].findAll('tr')[1:]
    for i in range(len(standard_fields_soup)):
        standard_field = re.sub("\d+|%|\.|^\s+|\n|\s+$", '', standard_fields_soup[i].text)
        if len(standard_field) > 1:
            standard_fields.append(standard_field)
    return standard_fields

In [7]:
standard_fields_titles = get_standard_fields_titles(start_soup)

In [8]:
headers.extend(standard_fields_titles)

In [9]:
len(headers)

23

### Названия регионов и ссылки на их сайты

In [11]:
regions_soup = start_soup.findAll('table')[1].findAll('table')[7].find('tr').findAll('td')

In [12]:
len(regions_soup)

87

In [13]:
def get_names(soup):
    return [soup[i].find('a').text for i in range(len(soup))]

In [14]:
def get_links(soup):
    return [soup[i].find('a')['href'] for i in range(len(soup))]

In [15]:
regions_names = get_names(regions_soup)

In [16]:
regions_links = get_links(regions_soup)

### Названия ТИК и ссылки на их сайты

In [19]:
def get_TIK_soup(reg_link):
    reg_req = requests.get(reg_link)
    reg_soup = BeautifulSoup(reg_req.text, 'lxml')
    TIK_soup = reg_soup.findAll('table')[1].findAll('table')[7].find('tr').findAll('td')
    return TIK_soup

def get_TIKs_soups(reg_links):
    TIKs_soups = []
    for i in range(len(reg_links)):
        TIK_soup = get_TIK_soup(reg_links[i])
        TIKs_soups.append(TIK_soup)
    return TIKs_soups

In [20]:
def get_TIKs_names(soups, reg_links, reg_names):
    TIKs_names = []
    for i in range(len(soups)):
        TIK_names = get_names(soups[i])
        if TIK_names != []:
            TIKs_names.append(TIK_names)
        else:
            TIKs_names.append([reg_names[i]])
    return TIKs_names

In [21]:
def get_TIKs_links(soups, reg_links):
    TIKs_links = []
    for i in range(len(soups)):
        TIK_links = get_links(soups[i])
        if TIK_links != []:
            TIKs_links.append(TIK_links)
        else:
            TIKs_links.append([reg_links[i]])
    return TIKs_links

In [22]:
TIKs_soups = get_TIKs_soups(regions_links)

In [23]:
TIKs_names = get_TIKs_names(TIKs_soups, regions_links, regions_names)

In [24]:
TIKs_links = get_TIKs_links(TIKs_soups, regions_links)

### Ссылки на сайты с УИК

In [27]:
def get_UIK_link(link, UIK_links):
    tik_req = requests.get(link)
    tik_soup = BeautifulSoup(tik_req.text, 'lxml')
    UIK_link = tik_soup.findAll('table')[1].findAll('tr')[1].find('td').find('a')['href']
    UIK_links.append(UIK_link)
    return UIK_links

def get_UIK_links(link):
    UIK_links = []
    for j in range(len(link)):
        UIK_links = get_UIK_link(link[j], UIK_links)
    return UIK_links

def get_UIKs_links(links):
    UIKs_links = []
    for i in range(len(links)):
        UIK_links = get_UIK_links(links[i])
        UIKs_links.append(UIK_links)
    return UIKs_links

In [28]:
UIKs_links = get_UIKs_links(TIKs_links)

### Сбор названий и статистики по УИК

In [40]:
def get_UIK_soup(link):
    uik_req = requests.get(link)
    uik_soup = BeautifulSoup(uik_req.text, 'lxml')
    return uik_soup

In [41]:
def get_table_soup(soup, count):
    if (count in [2, 16, 20, 37, 47, 81]):
        table = soup.findAll('table')[10].find('div').findAll('tr')
    elif (count in [6, 11, 13, 30, 34, 45, 58, 65, 72, 76, 79]):
        table = soup.find('div').find_all('tr') 
    elif (count in [36]):
        table = soup.find_all('div')[11].find_all('tr')
    elif (count in [42]):
        table = []
    elif (count in [82]):
        table = soup.find_all('div')[1].find_all('tr')
    else:
        table = soup.findAll('table')[2].findAll('table')[5].find('tr').find('div').findAll('tr')
    return table

In [42]:
def get_UIK_names(table, count): 
    UIK_names = []
    if (count != 42):
        names_soup = table[0].findAll('nobr')
        for name in names_soup:
            UIK_names.append(name.text)
    return UIK_names

In [43]:
def get_UIK_statistics_soup(table):
    uik_statistics_soup = []
    for row in table[1:]:
        uik_statistics_soup.append(row.findAll('nobr'))
    return uik_statistics_soup

def get_UIK_statistics(table, count):
    UIK_statistics = []
    if (count != 42):
        uik_statistics_soup = get_UIK_statistics_soup(table)
        for row in uik_statistics_soup:
            statistics_row = []
            for el in row:
                statistics_row.append(el.text)
            if (len(statistics_row) > 0):
                UIK_statistics.append(statistics_row)
    UIK_statistics_transpose = np.array(UIK_statistics).transpose()
    return UIK_statistics_transpose

### Формирование результирующей таблицы 

In [51]:
def get_UIK_row(reg_ind, tik_ind, uik_ind, UIK_names, UIK_statistics):
    UIK_row = [regions_names[reg_ind]]
    UIK_row.append(TIKs_names[reg_ind][tik_ind])
    UIK_row.append(UIK_names[uik_ind])
    for st_num in UIK_statistics[uik_ind]:
        UIK_row.append(st_num)
    return UIK_row

In [52]:
def get_UIK_lines_group_by_TIK(reg_ind, tik_ind):
    UIK_soup = get_UIK_soup(UIKs_links[reg_ind][tik_ind])
    UIK_table_soup = get_table_soup(UIK_soup, reg_ind)
    UIK_names = get_UIK_names(UIK_table_soup, reg_ind)
    UIK_statistics = get_UIK_statistics(UIK_table_soup, reg_ind)
    
    UIK_lines_group_by_TIK = []
    for uik_ind in range(len(UIK_names)):
        UIK_row = get_UIK_row(reg_ind, tik_ind, uik_ind, UIK_names, UIK_statistics)
        UIK_lines_group_by_TIK.append(UIK_row)
    return UIK_lines_group_by_TIK

In [124]:
def get_UIK_lines_group_by_region(reg_ind):
    UIK_lines_group_by_region = []
    for tik_ind in range(len(TIKs_names[reg_ind])):
        TIK_rows = get_UIK_lines_group_by_TIK(reg_ind, tik_ind)
        for UIK_line in TIK_rows:
            UIK_lines_group_by_region.append(UIK_line)
    return UIK_lines_group_by_region

In [147]:
def get_all_UIK_lines():
    all_UIK_lines = []
    for reg_ind in range(len(regions_names)):
        regions_rows = get_UIK_lines_group_by_region(reg_ind)
        for UIK_line in regions_rows:
            all_UIK_lines.append(UIK_line)
    return all_UIK_lines

In [148]:
UIKs_table = get_all_UIK_lines()

### Запись таблицы в csv-файл

In [150]:
def write_table_to_csv_file(file_name, table):
    df = pd.DataFrame(data = table, columns = headers)
    df.to_csv(file_name, index = False)

In [151]:
data_file = 'data.csv'

In [152]:
write_table_to_csv_file(data_file, UIKs_table)

In [153]:
pd.read_csv(data_file)

,Регион,ТИК,УИК,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,Республика Адыгея (Адыгея),Адыгейская,УИК №1,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
1,Республика Адыгея (Адыгея),Адыгейская,УИК №2,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
2,Республика Адыгея (Адыгея),Адыгейская,УИК №3,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
3,Республика Адыгея (Адыгея),Адыгейская,УИК №4,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
4,Республика Адыгея (Адыгея),Адыгейская,УИК №5,700,714,0,676,11,27,11,...,0,0,2,44,6,624,6,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97115,Территория за пределами РФ,Территория за пределами РФ,УИК №8433,436,2500,0,431,5,2064,5,...,0,0,3,32,13,371,8,3,3,3
97116,Территория за пределами РФ,Территория за пределами РФ,УИК №8434,1141,3000,0,1136,5,1859,5,...,0,0,1,10,5,1072,8,1,0,0
97117,Территория за пределами РФ,Территория за пределами РФ,УИК №8436,78,1507,0,78,0,1429,0,...,0,0,0,17,3,43,10,0,3,2
97118,Территория за пределами РФ,Территория за пределами РФ,УИК №8437,848,2500,0,743,105,1652,105,...,0,0,4,4,0,802,1,1,0,0
